* chrome driver 다운로드  
http://chromedriver.chromium.org/downloads

In [1]:
import pandas as pd
import numpy as np
import glob
import os

from bs4 import BeautifulSoup
from selenium import webdriver

In [2]:
# *****************************
# 파라미터 설정
# *****************************
# directory 설정
readDir = 'C:/Users/jseung21/Documents/CybosPlus/data'
writeDir = 'C:/Users/jseung21/Documents/CybosPlus/data/financeReport/'

In [3]:
# Chrome의 경우 | 아까 받은 chromedriver의 위치를 지정해준다.
driver = webdriver.Chrome('C:/Users/jseung21/Documents/CybosPlus/chromedriver_win32/chromedriver')
driver.implicitly_wait(3)

# 화면 이동
url_itooza = 'https://login.itooza.com/login.htm?qurl64=aHR0cDovL3d3dy5pdG9vemEuY29tLw=='
driver.get(url_itooza)

# 아이디/비밀번호를 입력해준다.
driver.find_element_by_name('txtUserId').send_keys('jseung21')
driver.find_element_by_name('txtPassword').send_keys('goddns21')

# 로그인 버튼을 눌러주자.
driver.find_element_by_css_selector('div.btn-login>input').click()

In [4]:
# *****************************
# 재무 보고서 생성
# *****************************
def getFinanceReport(_fileName, _url, _dir):
    # csv 파일명
    fineName = "{}_{}_{}.csv".format(_fileName, stockCode, stockName)

    # url
    url = _url.format(stockCode=stockCode)
#     print('url:[{}]'.format(url))
    # 페이지 이동
    driver.get(url)
    
    try:
        html = driver.page_source
        soup = BeautifulSoup(html, 'html.parser')

        # ***********************************
        # 항목
        # ***********************************
        _columns=soup.select('div.a01 > table > tbody > tr > th')
        columns = []
        for n in _columns:
            columns.append(n.text)

        # ***********************************
        # 분기날짜
        # ***********************************
        _dates = soup.select('table[id=y10_tb_2] > thead > tr > th')
        del _dates[0]
        dates = []
        for n in _dates:
            dates.append(n.text)

        # ***********************************
        # 재무 data
        # ***********************************
        data = soup.select('table[id=y10_tb_2] > tbody > tr > td')

        # 재무 데이터 재정리
        length = len(dates)
        j = 0
        listA = []
        for i in range (0, int(len(data)/length)):
            l = []
            for n in data[j:j+length]:
                l.append(n.text)
            listA.append(l)
            j += length

        # 데이터 dataFarme 생성
        df_convertData = pd.DataFrame(listA, columns=dates, index=columns)

        # csv 생성
        createFile = "{}/{}".format(writeDir+_dir,fineName) 
        df_convertData.to_csv(createFile, index=True, encoding='utf-8')

        print("[{}], [{}] 생성완료".format(index, fineName))
    except Exception as ex:
        print("[{}], [{}] 생성실패 !!!".format(index, fineName))
        driver.switch_to_alert().dismiss()
#         print('에러가 발생 했습니다', ex)    

In [5]:
# *****************************
# url 정보
# *****************************
# 제무상태표 분기 FinancialStatementsQuarter
urlFSQ = "http://www.itooza.com/vclub/y10_page.php?cmp_cd={stockCode}&mode=db&ss=10&sv=1&lsmode=1&lkmode=2&pmode=1&exmode=1&accmode=1"
# 손익계산서 분기 IncomeStatementQuarter
urlISQ = "http://www.itooza.com/vclub/y10_page.php?cmp_cd={stockCode}&mode=db&ss=10&sv=2&lsmode=1&lkmode=2&pmode=1&exmode=1&accmode=1"
# 손익계산서 연환산
urlISYS = "http://www.itooza.com/vclub/y10_page.php?cmp_cd={stockCode}&mode=dcy&ss=10&sv=2"
# 현금흐름표 분기 CashFlowsStatementQuarter
urlCFSQ = "http://www.itooza.com/vclub/y10_page.php?cmp_cd={stockCode}&mode=db&ss=10&sv=4&lsmode=1&lkmode=2&pmode=1&exmode=1&accmode=1"
# 현금흐름표_연환산
urlCFSYS = "http://www.itooza.com/vclub/y10_page.php?cmp_cd={stockCode}&mode=dcy&ss=10&sv=4"

# 재무비율/가치평가 연환산 financail ratio / value assessment year smooth
urlFRVAYS = "http://www.itooza.com/vclub/y10_page.php?cmp_cd={stockCode}&ss=10&sv=10"
# 재무비율/가치평가 분기
urlFRVAQ = "http://www.itooza.com/vclub/y10_page.php?ss=10&sv=10&cmp_cd={stockCode}&mode=db"
# 투자지표 연환산 Investment Index Year Smmoth
urlIIYS = "http://search.itooza.com/search.htm?seName={stockCode}&jl=&search_ck=&sm=&sd=&ed=&ds_de=&page=&cpv=#indexTable1"
# 투자지표 분기
urlIIQ = "http://search.itooza.com/search.htm?seName={stockCode}&jl=&search_ck=&sm=&sd=&ed=&ds_de=&page=&cpv=#indexTable3"


### loop

In [6]:
# 디렉토리안 csv 파일 가지고 오기
globbed_files = glob.glob(readDir+'/전종목.csv') #creates a list of all csv files

df_data = pd.read_csv(globbed_files[0], engine='python', encoding = 'utf8')

for index, row in df_data.iterrows():
    stockCode = row['stockCode']
    stockName = row['stockName']

    # 제무제표 분기 파일 생성
#     getFinanceReport("재무상태표", urlFSQ, '재무상태표')
    
    # 손익계산서 분기 파일 생성
#     getFinanceReport("손익계산서_분기", urlISQ, '손익계산서_분기')
    
    # 손익계산서 연환산 파일 생성
    getFinanceReport("손익계산서_연환산", urlISYS, '손익계산서_연환산')
    
    # 현금흐름표 분기 
#     getFinanceReport("현금흐름표_분기", urlCFSQ, '현금흐름표_분기')

    # 현금흐름표_연환산 파일 생성
    getFinanceReport("현금흐름표_연환산", urlCFSYS, '현금흐름표_연환산')



print('파일생성 종료!!!')
    
        


[0], [손익계산서_연환산_000020_동화약품.csv] 생성완료
[0], [현금흐름표_연환산_000020_동화약품.csv] 생성완료
[1], [손익계산서_연환산_000030_우리은행.csv] 생성완료
[1], [현금흐름표_연환산_000030_우리은행.csv] 생성완료
[2], [손익계산서_연환산_000040_KR모터스.csv] 생성완료
[2], [현금흐름표_연환산_000040_KR모터스.csv] 생성완료
[3], [손익계산서_연환산_000050_경방.csv] 생성완료
[3], [현금흐름표_연환산_000050_경방.csv] 생성완료
[4], [손익계산서_연환산_000060_메리츠화재.csv] 생성완료
[4], [현금흐름표_연환산_000060_메리츠화재.csv] 생성완료
[5], [손익계산서_연환산_000070_삼양홀딩스.csv] 생성완료
[5], [현금흐름표_연환산_000070_삼양홀딩스.csv] 생성완료
[6], [손익계산서_연환산_000075_삼양홀딩스우.csv] 생성실패 !!!


C:\Users\jseung21\Anaconda3\lib\site-packages\ipykernel_launcher.py:61: DeprecationWarning: use driver.switch_to.alert instead


[6], [현금흐름표_연환산_000075_삼양홀딩스우.csv] 생성실패 !!!
[7], [손익계산서_연환산_000080_하이트진로.csv] 생성완료
[7], [현금흐름표_연환산_000080_하이트진로.csv] 생성완료
[8], [손익계산서_연환산_000087_하이트진로2우B.csv] 생성실패 !!!
[8], [현금흐름표_연환산_000087_하이트진로2우B.csv] 생성실패 !!!
[9], [손익계산서_연환산_000100_유한양행.csv] 생성완료
[9], [현금흐름표_연환산_000100_유한양행.csv] 생성완료
[10], [손익계산서_연환산_000105_유한양행우.csv] 생성실패 !!!
[10], [현금흐름표_연환산_000105_유한양행우.csv] 생성실패 !!!
[11], [손익계산서_연환산_000120_CJ대한통운.csv] 생성완료
[11], [현금흐름표_연환산_000120_CJ대한통운.csv] 생성완료
[12], [손익계산서_연환산_000140_하이트진로홀딩스.csv] 생성완료
[12], [현금흐름표_연환산_000140_하이트진로홀딩스.csv] 생성완료
[13], [손익계산서_연환산_000145_하이트진로홀딩스우.csv] 생성실패 !!!
[13], [현금흐름표_연환산_000145_하이트진로홀딩스우.csv] 생성실패 !!!
[14], [손익계산서_연환산_000150_두산.csv] 생성완료
[14], [현금흐름표_연환산_000150_두산.csv] 생성완료
[15], [손익계산서_연환산_000155_두산우.csv] 생성실패 !!!
[15], [현금흐름표_연환산_000155_두산우.csv] 생성실패 !!!
[16], [손익계산서_연환산_000157_두산2우B.csv] 생성실패 !!!
[16], [현금흐름표_연환산_000157_두산2우B.csv] 생성실패 !!!
[17], [손익계산서_연환산_000180_성창기업지주.csv] 생성완료
[17], [현금흐름표_연환산_000180_성창기업지주.csv] 생성완료
[18], [손익계산서_연환산_000210_대림산업.c

[107], [손익계산서_연환산_001560_제일연마.csv] 생성완료
[107], [현금흐름표_연환산_001560_제일연마.csv] 생성완료
[108], [손익계산서_연환산_001570_금양.csv] 생성완료
[108], [현금흐름표_연환산_001570_금양.csv] 생성완료
[109], [손익계산서_연환산_001620_동국실업.csv] 생성완료
[109], [현금흐름표_연환산_001620_동국실업.csv] 생성완료
[110], [손익계산서_연환산_001630_종근당홀딩스.csv] 생성완료
[110], [현금흐름표_연환산_001630_종근당홀딩스.csv] 생성완료
[111], [손익계산서_연환산_001680_대상.csv] 생성완료
[111], [현금흐름표_연환산_001680_대상.csv] 생성완료
[112], [손익계산서_연환산_001685_대상우.csv] 생성실패 !!!
[112], [현금흐름표_연환산_001685_대상우.csv] 생성실패 !!!
[113], [손익계산서_연환산_001720_신영증권.csv] 생성완료
[113], [현금흐름표_연환산_001720_신영증권.csv] 생성완료
[114], [손익계산서_연환산_001725_신영증권우.csv] 생성실패 !!!
[114], [현금흐름표_연환산_001725_신영증권우.csv] 생성실패 !!!
[115], [손익계산서_연환산_001740_SK네트웍스.csv] 생성완료
[115], [현금흐름표_연환산_001740_SK네트웍스.csv] 생성완료
[116], [손익계산서_연환산_001745_SK네트웍스우.csv] 생성실패 !!!
[116], [현금흐름표_연환산_001745_SK네트웍스우.csv] 생성실패 !!!
[117], [손익계산서_연환산_001750_한양증권.csv] 생성완료
[117], [현금흐름표_연환산_001750_한양증권.csv] 생성완료
[118], [손익계산서_연환산_001755_한양증권우.csv] 생성실패 !!!
[118], [현금흐름표_연환산_001755_한양증권우.csv] 생성실패 !!!


[207], [현금흐름표_연환산_003340_SK건설.csv] 생성실패 !!!
[208], [손익계산서_연환산_003350_한국화장품제조.csv] 생성완료
[208], [현금흐름표_연환산_003350_한국화장품제조.csv] 생성완료
[209], [손익계산서_연환산_003380_하림지주.csv] 생성완료
[209], [현금흐름표_연환산_003380_하림지주.csv] 생성완료
[210], [손익계산서_연환산_003410_쌍용양회.csv] 생성완료
[210], [현금흐름표_연환산_003410_쌍용양회.csv] 생성완료
[211], [손익계산서_연환산_003415_쌍용양회우.csv] 생성실패 !!!
[211], [현금흐름표_연환산_003415_쌍용양회우.csv] 생성실패 !!!
[212], [손익계산서_연환산_003460_유화증권.csv] 생성완료
[212], [현금흐름표_연환산_003460_유화증권.csv] 생성완료
[213], [손익계산서_연환산_003465_유화증권우.csv] 생성실패 !!!
[213], [현금흐름표_연환산_003465_유화증권우.csv] 생성실패 !!!
[214], [손익계산서_연환산_003470_유안타증권.csv] 생성완료
[214], [현금흐름표_연환산_003470_유안타증권.csv] 생성완료
[215], [손익계산서_연환산_003475_유안타증권우.csv] 생성실패 !!!
[215], [현금흐름표_연환산_003475_유안타증권우.csv] 생성실패 !!!
[216], [손익계산서_연환산_003480_한진중공업홀딩스.csv] 생성완료
[216], [현금흐름표_연환산_003480_한진중공업홀딩스.csv] 생성완료
[217], [손익계산서_연환산_003490_대한항공.csv] 생성완료
[217], [현금흐름표_연환산_003490_대한항공.csv] 생성완료
[218], [손익계산서_연환산_003495_대한항공우.csv] 생성실패 !!!
[218], [현금흐름표_연환산_003495_대한항공우.csv] 생성실패 !!!
[219], [손익계산서_연환산_

...

[2534], [현금흐름표_연환산_252410_KBSTAR200선물인버스.csv] 생성완료
[2535], [손익계산서_연환산_252420_KBSTAR200선물인버스2X.csv] 생성완료
[2535], [현금흐름표_연환산_252420_KBSTAR200선물인버스2X.csv] 생성완료
[2536], [손익계산서_연환산_252500_세화피앤씨.csv] 생성완료
[2536], [현금흐름표_연환산_252500_세화피앤씨.csv] 생성완료
[2537], [손익계산서_연환산_252650_KODEX200동일가중.csv] 생성완료
[2537], [현금흐름표_연환산_252650_KODEX200동일가중.csv] 생성완료
[2538], [손익계산서_연환산_252670_KODEX200선물인버스2X.csv] 생성완료
[2538], [현금흐름표_연환산_252670_KODEX200선물인버스2X.csv] 생성완료
[2539], [손익계산서_연환산_252710_TIGER200선물인버스2X.csv] 생성완료
[2539], [현금흐름표_연환산_252710_TIGER200선물인버스2X.csv] 생성완료
[2540], [손익계산서_연환산_252720_KBSTAR모멘텀밸류.csv] 생성완료
[2540], [현금흐름표_연환산_252720_KBSTAR모멘텀밸류.csv] 생성완료
[2541], [손익계산서_연환산_252730_KBSTAR모멘텀로우볼.csv] 생성완료
[2541], [현금흐름표_연환산_252730_KBSTAR모멘텀로우볼.csv] 생성완료
[2542], [손익계산서_연환산_252980_대영씨엔이.csv] 생성실패 !!!
[2542], [현금흐름표_연환산_252980_대영씨엔이.csv] 생성실패 !!!
[2543], [손익계산서_연환산_253150_ARIRANG200선물레버리지.csv] 생성완료
[2543], [현금흐름표_연환산_253150_ARIRANG200선물레버리지.csv] 생성완료
[2544], [손익계산서_연환산_253160_ARIRANG200선물인버스2X.csv] 생성완료
[2544],

[2623], [현금흐름표_연환산_266410_KODEX필수소비재.csv] 생성완료
[2624], [손익계산서_연환산_266420_KODEX헬스케어.csv] 생성완료
[2624], [현금흐름표_연환산_266420_KODEX헬스케어.csv] 생성완료
[2625], [손익계산서_연환산_266470_바이오인프라생명과학.csv] 생성실패 !!!
[2625], [현금흐름표_연환산_266470_바이오인프라생명과학.csv] 생성실패 !!!
[2626], [손익계산서_연환산_266550_ARIRANG중형주저변동50.csv] 생성완료
[2626], [현금흐름표_연환산_266550_ARIRANG중형주저변동50.csv] 생성완료
[2627], [손익계산서_연환산_266870_파워풀엑스.csv] 생성완료
[2627], [현금흐름표_연환산_266870_파워풀엑스.csv] 생성완료
[2628], [손익계산서_연환산_267060_명진홀딩스.csv] 생성완료
[2628], [현금흐름표_연환산_267060_명진홀딩스.csv] 생성완료
[2629], [손익계산서_연환산_267250_현대중공업지주.csv] 생성완료
[2629], [현금흐름표_연환산_267250_현대중공업지주.csv] 생성완료
[2630], [손익계산서_연환산_267260_현대일렉트릭.csv] 생성완료
[2630], [현금흐름표_연환산_267260_현대일렉트릭.csv] 생성완료
[2631], [손익계산서_연환산_267270_현대건설기계.csv] 생성완료
[2631], [현금흐름표_연환산_267270_현대건설기계.csv] 생성완료
[2632], [손익계산서_연환산_267290_경동도시가스.csv] 생성완료
[2632], [현금흐름표_연환산_267290_경동도시가스.csv] 생성완료
[2633], [손익계산서_연환산_267300_TIGER코스닥150로우볼.csv] 생성완료
[2633], [현금흐름표_연환산_267300_TIGER코스닥150로우볼.csv] 생성완료
[2634], [손익계산서_연환산_267320_교보7호스팩.csv] 생

[2710], [손익계산서_연환산_279530_KODEX고배당.csv] 생성완료
[2710], [현금흐름표_연환산_279530_KODEX고배당.csv] 생성완료
[2711], [손익계산서_연환산_279540_KODEX최소변동성.csv] 생성완료
[2711], [현금흐름표_연환산_279540_KODEX최소변동성.csv] 생성완료
[2712], [손익계산서_연환산_279600_미디어젠.csv] 생성완료
[2712], [현금흐름표_연환산_279600_미디어젠.csv] 생성완료
[2713], [손익계산서_연환산_280320_KINDEX미국4차산업인터넷(합성H).csv] 생성완료
[2713], [현금흐름표_연환산_280320_KINDEX미국4차산업인터넷(합성H).csv] 생성완료
[2714], [손익계산서_연환산_280360_롯데제과.csv] 생성완료
[2714], [현금흐름표_연환산_280360_롯데제과.csv] 생성완료
[2715], [손익계산서_연환산_280920_ARIRANG주도업종.csv] 생성완료
[2715], [현금흐름표_연환산_280920_ARIRANG주도업종.csv] 생성완료
[2716], [손익계산서_연환산_280930_KODEX미국러셀2000(H).csv] 생성완료
[2716], [현금흐름표_연환산_280930_KODEX미국러셀2000(H).csv] 생성완료
[2717], [손익계산서_연환산_280940_KODEX골드선물인버스(H).csv] 생성완료
[2717], [현금흐름표_연환산_280940_KODEX골드선물인버스(H).csv] 생성완료
[2718], [손익계산서_연환산_281310_바이오시네틱스.csv] 생성완료
[2718], [현금흐름표_연환산_281310_바이오시네틱스.csv] 생성완료
[2719], [손익계산서_연환산_281410_한국제6호스팩.csv] 생성완료
[2719], [현금흐름표_연환산_281410_한국제6호스팩.csv] 생성완료
[2720], [손익계산서_연환산_281740_동부스팩5호.csv] 생성완료
[2720], [현금흐름

[2796], [현금흐름표_연환산_293180_HANARO200.csv] 생성완료
[2797], [손익계산서_연환산_293290_아이엔오기술.csv] 생성실패 !!!
[2797], [현금흐름표_연환산_293290_아이엔오기술.csv] 생성실패 !!!
[2798], [손익계산서_연환산_293480_하나제약.csv] 생성완료
[2798], [현금흐름표_연환산_293480_하나제약.csv] 생성완료
[2799], [손익계산서_연환산_293580_나우아이비캐피탈.csv] 생성완료
[2799], [현금흐름표_연환산_293580_나우아이비캐피탈.csv] 생성완료
[2800], [손익계산서_연환산_293940_신한알파리츠.csv] 생성완료
[2800], [현금흐름표_연환산_293940_신한알파리츠.csv] 생성완료
[2801], [손익계산서_연환산_294400_KOSEF200TR.csv] 생성완료
[2801], [현금흐름표_연환산_294400_KOSEF200TR.csv] 생성완료
[2802], [손익계산서_연환산_294870_HDC현대산업개발.csv] 생성완료
[2802], [현금흐름표_연환산_294870_HDC현대산업개발.csv] 생성완료
[2803], [손익계산서_연환산_295000_KBSTAR국채선물10년.csv] 생성완료
[2803], [현금흐름표_연환산_295000_KBSTAR국채선물10년.csv] 생성완료
[2804], [손익계산서_연환산_295020_KBSTAR국채선물10년인버스.csv] 생성완료
[2804], [현금흐름표_연환산_295020_KBSTAR국채선물10년인버스.csv] 생성완료
[2805], [손익계산서_연환산_295040_SMART200TR.csv] 생성완료
[2805], [현금흐름표_연환산_295040_SMART200TR.csv] 생성완료
[2806], [손익계산서_연환산_295820_ARIRANG200동일가중.csv] 생성완료
[2806], [현금흐름표_연환산_295820_ARIRANG200동일가중.csv] 생성완료
[2807], [손익계산서

[2882], [현금흐름표_연환산_309170_ARIRANGKRX300IT.csv] 생성완료
[2883], [손익계산서_연환산_309180_ARIRANGKRX300금융.csv] 생성완료
[2883], [현금흐름표_연환산_309180_ARIRANGKRX300금융.csv] 생성완료
[2884], [손익계산서_연환산_309190_ARIRANGKRX300산업재.csv] 생성완료
[2884], [현금흐름표_연환산_309190_ARIRANGKRX300산업재.csv] 생성완료
[2885], [손익계산서_연환산_309200_ARIRANGKRX300자유소비재.csv] 생성완료
[2885], [현금흐름표_연환산_309200_ARIRANGKRX300자유소비재.csv] 생성완료
[2886], [손익계산서_연환산_309210_ARIRANGKRX300헬스케어.csv] 생성완료
[2886], [현금흐름표_연환산_309210_ARIRANGKRX300헬스케어.csv] 생성완료
[2887], [손익계산서_연환산_309230_KINDEX미국WideMoat가치주.csv] 생성완료
[2887], [현금흐름표_연환산_309230_KINDEX미국WideMoat가치주.csv] 생성완료
[2888], [손익계산서_연환산_309930_삼성머스트스팩3호.csv] 생성완료
[2888], [현금흐름표_연환산_309930_삼성머스트스팩3호.csv] 생성완료
[2889], [손익계산서_연환산_310080_KBSTAR중국MSCIChina선물(H).csv] 생성완료
[2889], [현금흐름표_연환산_310080_KBSTAR중국MSCIChina선물(H).csv] 생성완료
[2890], [손익계산서_연환산_310200_미래에셋대우스팩2호.csv] 생성완료
[2890], [현금흐름표_연환산_310200_미래에셋대우스팩2호.csv] 생성완료
[2891], [손익계산서_연환산_310840_엔에이치스팩13호.csv] 생성완료
[2891], [현금흐름표_연환산_310840_엔에이치스팩13호.csv] 생성완료
[2892], [손익계